# Run

In [ ]:
%pip install -r requirements.txt

In [1]:
%load_ext autoreload
%reload_ext autoreload
%debug
#use logger
import logging
import time
import utils.config as config
import utils.io_utils as io_utils
if __name__ == "__main__":
    try:
        #logging.basicConfig(level=logging.DEBUG)
        start_time = time.time()
        io_utils.extract_EaglePack_and_process(config.input_path)
        end_time = time.time()
        print(f"Total processing time: {end_time - start_time:.2f} seconds")
    finally:
        if config.use_LLM:
            config.llm_processor.stop()
    print("Processing completed")

ERROR:root:No traceback has been produced, nothing to debug.


Not using LLM


c:\Users\Daniel\CodingProjects\eagle_tools\.venv\lib\site-packages\torch\functional.py:513: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at C:\actions-runner\_work\pytorch\pytorch\builder\windows\pytorch\aten\src\ATen\native\TensorShape.cpp:3610.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]
Images Queued: 100%|██████████| 164/164 [00:03<00:00, 44.83it/s]


Total processing time: 4.34 seconds
Processing completed


# Pixel Art Stuff

In [ ]:
import os
from PIL import Image

def resize_images(input_folder, output_folder, target_size):
    # Create output folder if it doesn't exist
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)

    # Iterate through all files in the input folder
    for filename in os.listdir(input_folder):
        if filename.lower().endswith(('.png', '.jpg', '.jpeg', '.gif', '.bmp')):
            input_path = os.path.join(input_folder, filename)
            output_path = os.path.join(output_folder, filename)

            with Image.open(input_path) as img:
                # Resize the image using nearest neighbor interpolation
                resized_img = img.resize(target_size, Image.NEAREST)
                
                # Save the resized image
                resized_img.save(output_path)
            
            print(f"Resized {filename}")




# Set these variables
input_folder = r"P:\downloads\ezgif-7-da110e1150-png-273x273-sprite-png"
output_folder = os.path.join(input_folder, "resized")
target_size = (64, 64)  # Set your desired size here (width, height)

# Run the resize function
resize_images(input_folder, output_folder, target_size)